# Creating Swarm of agents using Strands Agents

## Understanding Multi-Agent Systems and Swarm Intelligence

An agent swarm is a collection of autonomous AI agents working together to solve complex problems through collaboration. Inspired by natural systems like ant colonies or bird flocks, agent swarms leverage collective intelligence where the combined output exceeds what any single agent could produce. By distributing tasks and sharing information, swarms can tackle complex problems more efficiently and effectively than individual agents working in isolation.

Multi-agent systems consist of multiple interacting intelligent agents within an environment. These systems enable:

- *Distributed Problem Solving*: Breaking complex tasks into subtasks for parallel processing
- *Information Sharing*: Agents exchange insights to build collective knowledge
- *Specialization*: Different agents focus on specific aspects of a problem
- *Redundancy*: Multiple agents working on similar tasks improve reliability
- *Emergent Intelligence*: The system exhibits capabilities beyond those of its individual components

Swarm intelligence emphasizes:

1. *Decentralized Control*: No single agent directs the entire system
2. *Local Interactions*: Agents primarily interact with nearby agents
3. *Simple Rules*: Individual agents follow relatively simple behaviors
4. *Emergent Complexity*: Complex system behavior emerges from simple agent interactions

## 1. Quick Start with Swarm tool

The Strands Agents SDK provides a built-in swarm tool that simplifies the implementation of multi-agent systems, offering a quick start for users. This tool implements the shared memory.

In [6]:
!pip install -r requirements.txt
!pip install PyPDF2

In [7]:
from strands import Agent
from strands_tools import swarm
import time
import PyPDF2
import json
import boto3
import yaml
import os
import logging


In [10]:
import PyPDF2

def read_file(file_path):
    """
    Read and extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text from the PDF
    """
    try:
        # Open the PDF file in binary read mode
        # with open(file_path, 'rb') as file:
        #     # Create a PDF reader object
        #     pdf_reader = PyPDF2.PdfReader(file)
        with open(file_path, 'r') as file:
            file = file.read()
            source_json = json.loads(file)
            
            # # Get number of pages
            # num_pages = len(pdf_reader.pages)
            # print(f"Total pages: {num_pages}")
            
            # # Extract text from each page
            # text = ""
            # for page_num in range(num_pages):
            #     page = pdf_reader.pages[page_num]
            #     text += page.extract_text()
                
            return source_json
    
    except FileNotFoundError:
        return "Error: The file was not found."
    except PyPDF2.errors.PdfReadError:
        return "Error: Invalid PDF file or the file is encrypted."
    except Exception as e:
        return f"Error: {str(e)}"



In [11]:
# Example usage
if __name__ == "__main__":
    file_path = "data/FNOL.json"  # Replace with your PDF file path
    fnol_report = read_file(file_path)
    print("fnol_report text:")
    print(fnol_report)

fnol_report text:
{'fnol': {'claimNumber': 'CL-2023-1156789', 'reportDate': '2023-11-21', 'reportTime': '14:35:00', 'reportMethod': 'Online', 'submittedBy': 'Policyholder'}, 'policyInformation': {'policyNumber': 'AUTO-P987654321', 'policyType': 'Auto Insurance', 'effectiveDate': '2023-01-15', 'expirationDate': '2024-01-15', 'insuranceProvider': 'Acme Insurance Company', 'coverageTypes': [{'type': 'Liability', 'limit': '$300,000/$500,000'}, {'type': 'Collision', 'deductible': '$500'}, {'type': 'Comprehensive', 'deductible': '$250'}, {'type': 'Medical Payments', 'limit': '$10,000'}, {'type': 'Uninsured/Underinsured Motorist', 'limit': '$250,000/$500,000'}, {'type': 'Rental Reimbursement', 'limit': '$40/day, $1,200 maximum'}, {'type': 'Roadside Assistance', 'included': True}]}, 'policyholder': {'firstName': 'John', 'lastName': 'Smith', 'dateOfBirth': '1985-06-12', 'address': {'street': '123 Main Street', 'city': 'Springfield', 'state': 'IL', 'zipCode': '62704'}, 'contactInformation': {'ho

In [19]:
from strands import Agent

# Create specialized agents
fnol_extraction = Agent(system_prompt="You extract and structure Claim data from FNOL form.", callback_handler=None)
review_damage_assessment = Agent(system_prompt="You identify damages and assess estimated repair cost based on Claim data.", callback_handler=None)
payout_generation = Agent(system_prompt="You create a Claim payout report.")

# Sequential workflow processing
def process_workflow(fnol_report):
    # Step 1: FNOL Processing
    fnol_extraction_results = fnol_extraction(f"process {fnol_report}")

    # Step 2: Damage Assessment
    review_damage_assessment_results = review_damage_assessment(f"Analyze these research findings: {fnol_extraction_results}")

    # Step 3: Payout Generation
    payout_generation_results = payout_generation(f"Create a report based on this analysis: {review_damage_assessment_results}")

    return payout_generation_results

In [20]:
process_claim = process_workflow(fnol_report)

# CLAIM PAYOUT REPORT

**Claim Number:** CL-2023-1156789  
**Vehicle:** 2020 Honda Accord EX-L

## DAMAGE ASSESSMENT
- Rear bumper damage
- Trunk damage
- Taillight damage
- Possible frame damage
- Vehicle Status: Not drivable

## ESTIMATED COSTS

| Category | Amount | Notes |
|----------|--------|-------|
| Total Repair Estimate | $4,500 | Awaiting formal adjuster inspection |
| Policyholder Deductible | $500 | Per collision coverage terms |
| **Insurance Payout (Repair)** | **$4,000** | Pending investigation outcome |

## ADDITIONAL COVERED EXPENSES

### Rental Reimbursement
- Provider: Enterprise Rent-A-Car
- Vehicle: 2022 Nissan Altima
- Daily Rate: $38 (within $40/day policy limit)
- Duration: 14 days
- **Total Rental Cost: $532** (Covered under rental reimbursement)

### Potential Medical Expenses
- Reported Injury: Neck pain/possible whiplash
- Medical Payments Coverage: $10,000
- **Medical Expenses: TBD** - Requires medical documentation

## CLAIM STATUS
- Vehicle Location: Spr

### Using the Workflow Tool

In [18]:
from strands import Agent
from strands_tools import workflow

# Create an agent with workflow capability
agent = Agent(tools=[workflow])

# Create a multi-agent workflow
agent.tool.workflow(
    action="create",
    workflow_id="FNOL_processing",
    tasks=[
        {
            "task_id": "fnol_extraction",
            "description": "Proccess FNOL forms Extract key data to process Claim",
            "system_prompt": f"""Process {fnol_report}You extract and structure Claim data from FNOL form.""",
            "priority": 5
        },
        {
            "task_id": "review_damage_assessment",
            "description": "Analyze Claim data, damages and assess estimated repair cost",
            "dependencies": ["fnol_extraction"],
            "system_prompt": "You identify damages and assess estimated repair cost based on Claim data.",
            "priority": 3
        },
        {
            "task_id": "payout_generation",
            "description": "Generate Claim payout",
            "dependencies": ["review_damage_assessment"],
            "system_prompt": "You create a Claim payout report.",
            "priority": 2
        }
    ]
)

# Execute workflow (parallel processing where possible)
agent.tool.workflow(action="start", 
                    workflow_id="FNOL_processing",
                    )

# Check results
status = agent.tool.workflow(action="status", workflow_id="data_analysis")

# FNOL Claim Data Extraction Summary

## Claim Information
- **Claim Number**: CL-2023-1156789
- **Report Date/Time**: November 21, 2023 at 14:35
- **Report Method**: Online
- **Submitted By**: Policyholder

## Policy Information
- **Policy Number**: AUTO-P987654321
- **Type**: Auto Insurance
- **Effective Date**: January 15, 2023
- **Expiration Date**: January 15, 2024
- **Provider**: Acme Insurance Company

## Policyholder Information
- **Name**: John Smith
- **DOB**: June 12, 1985
- **Contact**: 217-555-5678 (cell), john.smith@email.com (preferred)
- **Driver's License**: S530-4281-7924 (IL)

## Incident Details
- **Date/Time**: November 19, 2023 at 17:45
- **Location**: 456 Oak Avenue, Springfield, IL 62704
- **Weather**: Light Rain, Wet Roads
- **Police Report**: Yes - SPD-2023-11875
- **Incident Type**: Rear-end collision while stopped at red light

## Vehicle Information
- **Insured Vehicle**: 2020 Honda Accord EX-L
- **VIN**: 1HGCV2F35LA007149
- **Damage**: Rear bumper, trunk, 